In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

user_agent = {'User-agent': 'Mozilla/5.0'}

game_info_list = []  # List to store extracted game information

for page in range(0,4):  # Set the range of pages you want to scrape (start, finish) (exclusive)
    main_url = f'https://www.metacritic.com/browse/games/genre/date/turn-based/all?page={page}'
    main_response = requests.get(main_url, headers=user_agent)
    main_soup = BeautifulSoup(main_response.text, 'html.parser')

    game_elements = main_soup.find_all('td', class_='clamp-image-wrap')  # Find all game elements through URL found in the product image 
    for game_element in game_elements:
        game_url = 'https://www.metacritic.com' + game_element.find('a')['href']  # Extract game URL
        game_response = requests.get(game_url, headers=user_agent)
        game_soup = BeautifulSoup(game_response.text, 'html.parser')

        release_date_element = game_soup.find('li', class_='summary_detail release_data')  # Find release date element
        release_date = release_date_element.find('span', class_='data').text.strip() if release_date_element else ''  # Extract release date or set empty if not found

        title_element = game_soup.find('h1') # Find game title element
        title = title_element.text.strip() if title_element is not None else ''  # Extract title of the game or set empty if not found

        platform_element = game_soup.find('span', class_='platform')  # Find platform element
        platform = platform_element.text.strip() if platform_element else ''  # Extract platform or set empty if not found

        rating_value_element = game_soup.find('span', itemprop='ratingValue') # Find the metascore value
        rating_value = rating_value_element.text.strip() if rating_value_element else '' #Extract the metascore value or set empty if not found

        developer_element = game_soup.find('li', class_='summary_detail developer')  # Find developer studio(s)
        developer = developer_element.find('span', class_='data').text.strip() if developer_element else ''  # Extract developer studio(s) OR set empty '' if not found

        game_info = {'title': title, 'release_date': release_date, 'platform': platform, 'rating_value': rating_value, 'developer': developer}          # Append the different variables in the underlying columns
        game_info_list.append(game_info)

game_info_df = pd.DataFrame(game_info_list)  # Convert list of dictionaries to DataFrame

# Export DataFrame to CSV
game_info_df.to_csv('C:/Users/Trabala/Desktop/Scrape_bs4/tb_game.csv', index=False)
